# 6. AI Querying with Databricks SQL

Databricks SQL includes powerful AI functions that let you integrate large language models directly into your SQL queries. The `ai_query` function allows you to send natural language prompts to AI models and get responses back as part of your SQL workflow.

This notebook will show you simple, practical examples of how to use AI querying to enhance your data analysis.

## What is ai_query?

The `ai_query` function lets you call large language models (LLMs) directly from SQL. You can:

- ✨ **Analyze text data** with natural language prompts
- 🔍 **Extract insights** from unstructured content
- 📝 **Generate summaries** of data
- 🏷️ **Classify and categorize** information
- 💬 **Create conversational interfaces** over your data

### Basic Syntax:
```sql
SELECT ai_query('your_endpoint_name', 'Your prompt here')
```

📖 **Complete Reference:** [ai_query Function Documentation](https://docs.databricks.com/en/sql/language-manual/functions/ai_query.html)

## Prerequisites

Before you can use `ai_query`, you need:

1. **A model serving endpoint** deployed in your workspace
2. **Permissions** to access the endpoint
3. **Unity Catalog** enabled (recommended)

💡 **Tip:** Your workspace may already have foundation model endpoints available like `databricks-llama-2-70b-chat` or similar. Check with your admin or look in the "Serving" section of your workspace.

## Example 1: Simple Text Analysis

In [ ]:
%sql
-- Basic example: Analyze the sentiment of a text
SELECT ai_query(
  'databricks-llama-2-70b-chat',
  'Analyze the sentiment of this text and respond with just one word (positive, negative, or neutral): 
   "I love using Databricks for data analysis! It makes my work so much easier."'
) as sentiment_analysis

## Example 2: Data Classification

In [ ]:
%sql
-- Classify customer feedback into categories
WITH sample_feedback AS (
  SELECT 'The product arrived quickly and works perfectly!' as feedback
  UNION ALL
  SELECT 'Shipping was delayed and the package was damaged'
  UNION ALL  
  SELECT 'Great customer service, very helpful staff'
  UNION ALL
  SELECT 'The website is confusing and hard to navigate'
)
SELECT 
  feedback,
  ai_query(
    'databricks-llama-2-70b-chat',
    CONCAT('Classify this customer feedback into one category: Product, Shipping, Service, or Website. 
            Only respond with the category name: ', feedback)
  ) as category
FROM sample_feedback;

## Example 3: Data Summarization

In [ ]:
%sql
-- Create summaries of longer text content
WITH sample_reports AS (
  SELECT 'Q1 2024 Sales Report: Revenue increased by 15% compared to Q1 2023. 
          New customer acquisitions were up 22%, while customer retention remained stable at 89%. 
          Product line A performed exceptionally well with 35% growth, while Product line B saw a 5% decline. 
          Marketing spend increased by 10% with improved ROI of 3.2x.' as report_text
)
SELECT 
  ai_query(
    'databricks-llama-2-70b-chat',
    CONCAT('Summarize this business report in 2-3 bullet points: ', report_text)
  ) as summary
FROM sample_reports;

## Example 4: Working with Real Data

Let's use AI querying with actual data from your workspace. We'll use the bike trips data that was created during setup.

In [ ]:
%sql
-- Analyze patterns in bike trip data using AI
WITH trip_summary AS (
  SELECT 
    COUNT(*) as total_trips,
    AVG(trip_duration) as avg_duration_minutes,
    COUNT(DISTINCT user_type) as user_types
  FROM main.dbdemos_pipeline_bike.bike_trips_gold
  LIMIT 1
)
SELECT 
  ai_query(
    'databricks-llama-2-70b-chat',
    CONCAT('Analyze this bike sharing data and provide 2-3 insights: ',
           'Total trips: ', total_trips, 
           ', Average duration: ', ROUND(avg_duration_minutes, 1), ' minutes',
           ', User types: ', user_types)
  ) as data_insights
FROM trip_summary;

## Example 5: Creating Dynamic Reports

In [ ]:
%sql
-- Generate a natural language report from aggregated data
WITH monthly_stats AS (
  SELECT 
    DATE_FORMAT(start_time, 'yyyy-MM') as month,
    COUNT(*) as trip_count,
    AVG(trip_duration) as avg_duration,
    COUNT(DISTINCT start_station_name) as stations_used
  FROM main.dbdemos_pipeline_bike.bike_trips_gold
  WHERE start_time >= '2024-01-01'
  GROUP BY DATE_FORMAT(start_time, 'yyyy-MM')
  ORDER BY month DESC
  LIMIT 3
)
SELECT 
  month,
  trip_count,
  ROUND(avg_duration, 1) as avg_duration_minutes,
  ai_query(
    'databricks-llama-2-70b-chat',
    CONCAT('Write a brief business summary for this month: ',
           month, ' had ', trip_count, ' bike trips with average duration of ', 
           ROUND(avg_duration, 1), ' minutes across ', stations_used, ' stations.')
  ) as monthly_report
FROM monthly_stats;

## Best Practices for AI Querying

### 🎯 **Prompt Engineering Tips:**
1. **Be specific** in your prompts - clear instructions get better results
2. **Limit response length** when needed (e.g., "respond in one word")
3. **Provide context** about the data you're analyzing
4. **Use consistent formatting** for structured outputs

### ⚡ **Performance Considerations:**
1. **Batch processing** - Process multiple rows efficiently rather than single calls
2. **Cache results** when possible to avoid repeated API calls
3. **Filter data first** - Apply WHERE clauses before using ai_query
4. **Monitor costs** - AI model calls consume compute resources

### 🔒 **Security Notes:**
1. **Don't send sensitive data** to external models unless approved
2. **Use appropriate endpoints** - some models may have data residency requirements
3. **Review outputs** - AI responses should be validated for business use

## Common Use Cases

### 📊 **Analytics & Reporting:**
- Generate natural language summaries of KPIs
- Create automated insights from trend analysis
- Explain complex data patterns in simple terms

### 🏷️ **Data Processing:**
- Classify and categorize text data
- Extract entities from unstructured content
- Standardize data formats and values

### 💼 **Business Intelligence:**
- Create conversational dashboards
- Generate executive summaries
- Provide context for data visualizations

### 🔍 **Data Quality:**
- Detect anomalies in text data
- Validate data consistency
- Identify potential data issues

## Troubleshooting Common Issues

### Model Endpoint Not Found:
```sql
-- Check available endpoints
SHOW ENDPOINTS;
```

### Permission Issues:
- Ensure you have `CAN QUERY` permissions on the model endpoint
- Contact your workspace admin if you can't access endpoints

### Timeout Errors:
- Simplify your prompts
- Process smaller batches of data
- Consider using asynchronous processing for large datasets

### Unexpected Responses:
- Review and refine your prompt wording
- Add more specific instructions
- Test with smaller datasets first

## Next Steps

### 🚀 **Explore Further:**
1. **Try different models** - Experiment with various foundation models
2. **Build applications** - Create notebooks that combine AI with your data workflows  
3. **Create dashboards** - Use AI-generated insights in your visualizations
4. **Automate reports** - Schedule jobs that include AI analysis

### 📚 **Learn More:**
* [Model Serving Documentation](https://docs.databricks.com/en/machine-learning/model-serving/index.html)
* [Foundation Models on Databricks](https://docs.databricks.com/en/machine-learning/foundation-models/index.html)
* [Generative AI Solutions](https://docs.databricks.com/en/generative-ai/index.html)

### 💡 **Community Resources:**
* [Databricks Community - AI/ML](https://community.databricks.com/s/topic/0TO5w00000094LGGAY/generative-ai)
* [SQL AI Functions Examples](https://docs.databricks.com/en/sql/language-manual/sql-ref-functions.html#ai-functions)

# 6. AI Querying with Databricks SQL

Databricks SQL includes powerful AI functions that let you integrate large language models directly into your SQL queries. The `ai_query` function allows you to send natural language prompts to AI models and get responses back as part of your SQL workflow.

This notebook will show you simple, practical examples of how to use AI querying to enhance your data analysis.

## What is ai_query?

The `ai_query` function lets you call large language models (LLMs) directly from SQL. You can:

- ✨ **Analyze text data** with natural language prompts
- 🔍 **Extract insights** from unstructured content
- 📝 **Generate summaries** of data
- 🏷️ **Classify and categorize** information
- 💬 **Create conversational interfaces** over your data

### Basic Syntax:
```sql
SELECT ai_query('your_endpoint_name', 'Your prompt here')
```

📖 **Complete Reference:** [ai_query Function Documentation](https://docs.databricks.com/en/sql/language-manual/functions/ai_query.html)

## Prerequisites

Before you can use `ai_query`, you need:

1. **A model serving endpoint** deployed in your workspace
2. **Permissions** to access the endpoint
3. **Unity Catalog** enabled (recommended)

💡 **Tip:** Your workspace may already have foundation model endpoints available like `databricks-llama-2-70b-chat` or similar. Check with your admin or look in the "Serving" section of your workspace.

## Example 1: Simple Text Analysis

In [ ]:
%sql
-- Basic example: Analyze the sentiment of a text
SELECT ai_query(
  'databricks-llama-2-70b-chat',
  'Analyze the sentiment of this text and respond with just one word (positive, negative, or neutral): 
   "I love using Databricks for data analysis! It makes my work so much easier."'
) as sentiment_analysis

## Example 2: Data Classification

In [ ]:
%sql
-- Classify customer feedback into categories
WITH sample_feedback AS (
  SELECT 'The product arrived quickly and works perfectly!' as feedback
  UNION ALL
  SELECT 'Shipping was delayed and the package was damaged'
  UNION ALL  
  SELECT 'Great customer service, very helpful staff'
  UNION ALL
  SELECT 'The website is confusing and hard to navigate'
)
SELECT 
  feedback,
  ai_query(
    'databricks-llama-2-70b-chat',
    CONCAT('Classify this customer feedback into one category: Product, Shipping, Service, or Website. 
            Only respond with the category name: ', feedback)
  ) as category
FROM sample_feedback;

## Example 3: Data Summarization

In [ ]:
%sql
-- Create summaries of longer text content
WITH sample_reports AS (
  SELECT 'Q1 2024 Sales Report: Revenue increased by 15% compared to Q1 2023. 
          New customer acquisitions were up 22%, while customer retention remained stable at 89%. 
          Product line A performed exceptionally well with 35% growth, while Product line B saw a 5% decline. 
          Marketing spend increased by 10% with improved ROI of 3.2x.' as report_text
)
SELECT 
  ai_query(
    'databricks-llama-2-70b-chat',
    CONCAT('Summarize this business report in 2-3 bullet points: ', report_text)
  ) as summary
FROM sample_reports;

## Example 4: Working with Real Data

Let's use AI querying with actual data from your workspace. We'll use the bike trips data that was created during setup.

In [ ]:
%sql
-- Analyze patterns in bike trip data using AI
WITH trip_summary AS (
  SELECT 
    COUNT(*) as total_trips,
    AVG(trip_duration) as avg_duration_minutes,
    COUNT(DISTINCT user_type) as user_types
  FROM main.dbdemos_pipeline_bike.bike_trips_gold
  LIMIT 1
)
SELECT 
  ai_query(
    'databricks-llama-2-70b-chat',
    CONCAT('Analyze this bike sharing data and provide 2-3 insights: ',
           'Total trips: ', total_trips, 
           ', Average duration: ', ROUND(avg_duration_minutes, 1), ' minutes',
           ', User types: ', user_types)
  ) as data_insights
FROM trip_summary;

## Example 5: Creating Dynamic Reports

In [ ]:
%sql
-- Generate a natural language report from aggregated data
WITH monthly_stats AS (
  SELECT 
    DATE_FORMAT(start_time, 'yyyy-MM') as month,
    COUNT(*) as trip_count,
    AVG(trip_duration) as avg_duration,
    COUNT(DISTINCT start_station_name) as stations_used
  FROM main.dbdemos_pipeline_bike.bike_trips_gold
  WHERE start_time >= '2024-01-01'
  GROUP BY DATE_FORMAT(start_time, 'yyyy-MM')
  ORDER BY month DESC
  LIMIT 3
)
SELECT 
  month,
  trip_count,
  ROUND(avg_duration, 1) as avg_duration_minutes,
  ai_query(
    'databricks-llama-2-70b-chat',
    CONCAT('Write a brief business summary for this month: ',
           month, ' had ', trip_count, ' bike trips with average duration of ', 
           ROUND(avg_duration, 1), ' minutes across ', stations_used, ' stations.')
  ) as monthly_report
FROM monthly_stats;

## Best Practices for AI Querying

### 🎯 **Prompt Engineering Tips:**
1. **Be specific** in your prompts - clear instructions get better results
2. **Limit response length** when needed (e.g., "respond in one word")
3. **Provide context** about the data you're analyzing
4. **Use consistent formatting** for structured outputs

### ⚡ **Performance Considerations:**
1. **Batch processing** - Process multiple rows efficiently rather than single calls
2. **Cache results** when possible to avoid repeated API calls
3. **Filter data first** - Apply WHERE clauses before using ai_query
4. **Monitor costs** - AI model calls consume compute resources

### 🔒 **Security Notes:**
1. **Don't send sensitive data** to external models unless approved
2. **Use appropriate endpoints** - some models may have data residency requirements
3. **Review outputs** - AI responses should be validated for business use

## Common Use Cases

### 📊 **Analytics & Reporting:**
- Generate natural language summaries of KPIs
- Create automated insights from trend analysis
- Explain complex data patterns in simple terms

### 🏷️ **Data Processing:**
- Classify and categorize text data
- Extract entities from unstructured content
- Standardize data formats and values

### 💼 **Business Intelligence:**
- Create conversational dashboards
- Generate executive summaries
- Provide context for data visualizations

### 🔍 **Data Quality:**
- Detect anomalies in text data
- Validate data consistency
- Identify potential data issues

## Troubleshooting Common Issues

### Model Endpoint Not Found:
```sql
-- Check available endpoints
SHOW ENDPOINTS;
```

### Permission Issues:
- Ensure you have `CAN QUERY` permissions on the model endpoint
- Contact your workspace admin if you can't access endpoints

### Timeout Errors:
- Simplify your prompts
- Process smaller batches of data
- Consider using asynchronous processing for large datasets

### Unexpected Responses:
- Review and refine your prompt wording
- Add more specific instructions
- Test with smaller datasets first

## Next Steps

### 🚀 **Explore Further:**
1. **Try different models** - Experiment with various foundation models
2. **Build applications** - Create notebooks that combine AI with your data workflows  
3. **Create dashboards** - Use AI-generated insights in your visualizations
4. **Automate reports** - Schedule jobs that include AI analysis

### 📚 **Learn More:**
* [Model Serving Documentation](https://docs.databricks.com/en/machine-learning/model-serving/index.html)
* [Foundation Models on Databricks](https://docs.databricks.com/en/machine-learning/foundation-models/index.html)
* [Generative AI Solutions](https://docs.databricks.com/en/generative-ai/index.html)

### 💡 **Community Resources:**
* [Databricks Community - AI/ML](https://community.databricks.com/s/topic/0TO5w00000094LGGAY/generative-ai)
* [SQL AI Functions Examples](https://docs.databricks.com/en/sql/language-manual/sql-ref-functions.html#ai-functions)